# Hate Speech Detector - EN - Features extraction for SVM & Dense model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
import syllables as sylla

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'svm'
dim = 10 if MODEL == 'svm' else 200

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/DavidsonEtAl/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/DavidsonEtAl/labeled_data.csv', 'r') as f:
        for d in tqdm(list(csv.reader(f))[1:]):
            tweets.append(d[6])  # tweet
            labels.append(d[5])  # class
    with open('hsd/DavidsonEtAl/perfect_data.pkl', 'w') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/DavidsonEtAl/perfect_data.pkl', 'r') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 24783
Labels: 24783


In [5]:
list(zip(tweets[:5], labels[:5]))

[("!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
  0),
 ('!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
  1),
 ('!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
  1),
 ('!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny', 1),
 ('!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;',
  1)]

## Features extraction

In [6]:
stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

sentiment_analyzer = VS()

In [7]:
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    #hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = re.sub(hashtag_regex, '', parsed_text)
    return parsed_text

def basic_tokenize(tweet):
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

def get_pos_string(tweet):
    text = preprocess(tweet)
    tokens = basic_tokenize(preprocess(text))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    tag_str = ' '.join(tag_list)
    
    return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = sylla.estimate(words)
    num_chars = sum(len(w) for w in words)
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59, 1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)), 2)
    
    twitter_objs = count_twitter_objs(tweet)
    retweet = 0 if "rt" in words else 1
    features = [FKRA, FRE, syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    return features

### Supervised fastText wordtokens training

In [8]:
if not os.path.exists('hsd/DavidsonEtAl/fasttext_{}.ft'.format(MODEL)):
    with open('hsd/DavidsonEtAl/fasttext_{}.ft'.format(MODEL), 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)))

# load fasttext model or train & save if none
if os.path.exists('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/DavidsonEtAl/fasttext_{}.ft'.format(MODEL),
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/DavidsonEtAl/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [9]:
def get_wordtoken_fts(data):
    
    sentences_words = []
    for d in tqdm(data):
        sentence = preprocess(d)
        sentences_words.append(sentence.split(' '))
    
    opt_length = int(np.median([len(sw) for sw in sentences_words]))
    sentences_words = [pad_words(sw, opt_length) for sw in sentences_words]
    
    ft_vectors = []
    for sw in tqdm(sentences_words):
        ft_vector = []
        for w in sw:
            ft_vector.extend(ft_model[w])
        ft_vectors.append(ft_vector)
    
    return ft_vectors

In [10]:
wordtoken_features = get_wordtoken_fts(tweets)

In [11]:
wordtoken_features[0]

[0.007928177,
 0.11992569,
 0.0028752065,
 0.030718988,
 0.062001467,
 -0.007022215,
 -0.07705958,
 0.08626743,
 0.018074136,
 0.08626623,
 0.16760364,
 -0.25004625,
 0.05675336,
 0.021296112,
 0.1535667,
 -0.07181231,
 0.19030358,
 0.04006586,
 -0.32047436,
 -0.23894314,
 0.090784736,
 -0.25651675,
 -0.09038265,
 -0.0018909777,
 -0.024543583,
 -0.014274004,
 0.1895897,
 -0.029224055,
 -0.1547373,
 -0.23869601,
 0.13214399,
 -0.03543946,
 -0.08781389,
 0.009753116,
 -0.031385317,
 -0.03915471,
 0.08630752,
 -0.06722911,
 -0.09088202,
 -0.05752908,
 -0.39256153,
 0.5459367,
 0.12314718,
 -0.108320236,
 -0.09030823,
 0.008959142,
 -0.11464913,
 0.015319895,
 0.09448529,
 0.034212455,
 -0.07821279,
 0.31698993,
 0.082358114,
 -0.07300558,
 -0.016081097,
 0.028319005,
 -0.07943164,
 -0.027838785,
 0.018876733,
 0.024743862,
 -0.7822932,
 1.3342228,
 0.1795538,
 -0.40651023,
 -0.21574245,
 -0.039143153,
 -0.37216437,
 -0.15515819,
 0.5185793,
 0.4386448,
 0.11361385,
 -0.2186819,
 0.0061534

### Supervised fastText pos training

In [12]:
if not os.path.exists('hsd/DavidsonEtAl/fasttext_pos_{}.ft'.format(MODEL)):
    with open('hsd/DavidsonEtAl/fasttext_pos_{}.ft'.format(MODEL), 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, get_pos_string(t)))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/DavidsonEtAl/fasttext_pos_{}.ft'.format(MODEL),
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/DavidsonEtAl/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [13]:
def get_pos_fts(data):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in tqdm(data):
        pos_string = get_pos_string(d)
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    opt_length = int(np.median([len(pt) for pt in pos_tags]))
    pos_tags = [pad_words(pt, opt_length) for pt in pos_tags]
    
    ft_vectors = []
    for pt in tqdm(pos_tags):
        ft_vector = []
        for t in pt:
            ft_vector.extend(ft_pos_model[t])
        ft_vectors.append(ft_vector)
    
    return ft_vectors

In [14]:
pos_features = get_pos_fts(tweets)

In [15]:
pos_features[0]

[-0.3302995,
 1.2387867,
 0.03511857,
 -0.003499597,
 0.023812015,
 0.036711995,
 0.5470143,
 -0.9115662,
 -0.036720674,
 0.23484771,
 -0.5209875,
 1.913564,
 -0.21366146,
 0.32891658,
 0.110604905,
 -0.24601156,
 0.6151405,
 0.18846886,
 0.5396054,
 0.62362736,
 -0.23190664,
 0.9580747,
 -0.22189823,
 0.3510105,
 0.34789371,
 0.43311724,
 0.14764233,
 -0.4856387,
 0.45577163,
 0.1420188,
 -0.17901443,
 1.4668449,
 -0.19256817,
 -0.019572409,
 -0.11394046,
 -0.14568205,
 0.15846448,
 -0.23181453,
 0.072660044,
 0.2347832,
 -0.5209875,
 1.913564,
 -0.21366146,
 0.32891658,
 0.110604905,
 -0.24601156,
 0.6151405,
 0.18846886,
 0.5396054,
 0.62362736,
 -1.1092862,
 1.9167979,
 -0.24071167,
 -0.12813602,
 -0.43814883,
 -0.20309842,
 0.64285403,
 0.13513944,
 0.22155532,
 1.4106488,
 0.33328894,
 -0.037182942,
 0.74806005,
 0.16574498,
 -0.3084814,
 -0.015700206,
 0.1743191,
 0.02270176,
 0.1213183,
 0.46741793,
 -0.10187656,
 0.64105576,
 -0.59103274,
 0.20252323,
 -0.08733121,
 -0.1538258

### Other features

In [16]:
other_features = np.array([other_features(t) for t in tqdm(tweets)])

In [17]:
other_features[:5]

array([[ 7.4000e+00,  8.6710e+01,  2.8000e+01,  1.1200e+00,  1.2700e+02,
         1.4000e+02,  2.5000e+01,  2.5000e+01,  2.3000e+01,  0.0000e+00,
         1.2000e-01,  8.8000e-01,  4.5630e-01,  0.0000e+00,  1.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 1.7000e+00,  1.1128e+02,  1.5000e+01,  9.3750e-01,  7.7000e+01,
         8.5000e+01,  1.6000e+01,  1.6000e+01,  1.6000e+01,  2.3700e-01,
         0.0000e+00,  7.6300e-01, -6.8760e-01,  0.0000e+00,  1.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 3.4000e+00,  1.0617e+02,  1.9000e+01,  9.5000e-01,  9.3000e+01,
         1.2000e+02,  2.1000e+01,  2.0000e+01,  1.8000e+01,  5.3800e-01,
         0.0000e+00,  4.6200e-01, -9.5500e-01,  0.0000e+00,  2.0000e+00,
         0.0000e+00,  0.0000e+00],
       [-2.1000e+00,  1.2469e+02,  7.0000e+00,  8.7500e-01,  3.8000e+01,
         6.2000e+01,  9.0000e+00,  8.0000e+00,  8.0000e+00,  0.0000e+00,
         3.4400e-01,  6.5600e-01,  5.6730e-01,  0.0000e+00,  2.0000e+00,
         0.0000e+00

### All features and feature names

In [18]:
#Now join them all up
features = np.concatenate([wordtoken_features, pos_features, other_features],axis=1)

In [19]:
features.shape

(24783, 267)

## Save features & labels

In [20]:
archive = dir_archive('hsd/DavidsonEtAl/X_y_{}'.format(MODEL), {'features': features, 'labels': labels}, serialized=True)
archive.dump()
del archive